In [1]:
from utils import prepare_pauli_operator,mean_and_error,parallel_for
from tqdm import tqdm
from math import sqrt
import networkx as nx
from functools import reduce # Valid in Python 2.6+, required in Python 3
import operator
from graphstates import *
import numpy as np
import itertools 
from utils import prepare_intertwined_grid_with_endpoints, prepare_intertwined_twist_with_endpoints
def prod(gen,init=1):
    return reduce(operator.mul, gen, init)
from tqdm import *
import pickle
import matplotlib as plt
from qutip.metrics import fidelity
from IPython.display import clear_output


def e_z_check(args):
    N,edges,Pedge,Pz,tomeasure,terminal,stabgens,axis=args
    gse=subgraph_and_z_one_error_ensemble(N,edges,Pedge,Pz)
    signs=[0 for v in range(N)]
    for v in tomeasure: #measure all non-terminal qubits
        sign=(1 if gse.perform_pauli_measurement(v,axis)=='+' else -1)
        signs[v]=sign
    ss=[]

    for sg in stabgens:
        ss.append(prod(signs[v] for v in sg))
    consistent=all(sign==1 for sign in ss) #check sign consistencies
    if consistent:
        st1=gse.get_reduced_state(terminal,normalized=True)
        st2=gse.states[-1]['state'].get_reduced_state(terminal,normalized=True)
        con=fidelity(st1,st2)
    else: #stabilizer-inconsistent outcomes postselected out.
        con=0
    return con,all(sign==1 for sign in ss)

def graphcheck(G,terminal,tomeasure,stabgens,no_measurement_runs,eps,draw=False):
    N=G.order()
    if draw:
        nx.draw(G,with_labels=True)
    edges=G.edges
    
    #generate random measurement results for edge loss:
    eresults=parallel_for(e_z_check,[(N,edges,eps,0,tomeasure,terminal,stabgens,'x') for _ in range(no_measurement_runs)])
    econs=[con for (con,passed) in eresults] # ... and extract just the fidelity
    epscons=[con for (con,passed) in eresults if passed] #... possibly after postselection on stab.-consistent outcomes

    #the same for Z-flip noise:
    vcons=[]
    vpscons=[]
    vresults=parallel_for(e_z_check,[(N,edges,0,eps,tomeasure,terminal,stabgens,'x') for _ in range(no_measurement_runs)])
    vcons=[con for (con,passed) in vresults]
    vpscons=[con for (con,passed) in vresults if passed]
    
    #determine the fidelity susceptibility for:
    esus=mean_and_error((1-np.array(econs))/eps) # edge-loss without postselection
    vsus=mean_and_error((1-np.array(vcons))/eps) # Z-flip without postselection
    epssus=mean_and_error((1-np.array(epscons))/eps) #edge loss with postselection
    vpssus=mean_and_error((1-np.array(vpscons))/eps) #Z-flip with postselection
    
    return (esus,vsus,epssus,vpssus)   


In [2]:
def pathcheck(length,eps,no_measurement_runs=300,clear=True,draw=False):
    G=nx.path_graph(length+2)
    N=G.order()
    terminal=[0,N-1]
    tomeasure=list(np.arange(1,N-1))
    stabgens=[]
    ret= graphcheck(G,terminal,tomeasure,stabgens,no_measurement_runs,eps,draw=draw)
    if clear:
        clear_output(wait=True)
    return ret
    
def crazycheck(length,eps,no_measurement_runs=300,clear=True,draw=False):
    G=prepare_intertwined_twist_with_endpoints(length)
    N=G.order()
    edges=G.edges
    cons={}
    postselcons={}
    terminal=[0,1]
    tomeasure=np.arange(2,N)
    stabgens=np.reshape(np.arange(2,N),((N-2)//2,2))
    ret= graphcheck(G,terminal,tomeasure,stabgens,no_measurement_runs,eps,draw=draw)
    if clear:
        clear_output(wait=True)
    return ret
def twistcheck(length,eps,no_measurement_runs=300,clear=True,draw=False):
    assert (length%2)==1
    G=prepare_intertwined_twist_with_endpoints(length) #prepare a crazy graph...
    for i in range(4,2*length,4): #... and contract nodes at every 2nd layer. easier to do it this way.
        G=nx.contracted_nodes(G,i,i+1,self_loops=False)
    G=nx.convert_node_labels_to_integers(G)
    N=G.order()
    edges=G.edges
    terminal=[0,1]
    tomeasure=list(range(2,G.order()))
    stabgens=[]
    for i in range(2,G.order(),3):
        stabgens.append([i,i+1])

    ret= graphcheck(G,terminal,tomeasure,stabgens,no_measurement_runs,eps,draw=draw)
    if clear:
        clear_output(wait=True)
    return ret

In [3]:
pathres={}
twistres={}
crazyres={}
eps=0.01

In [4]:
for l in range(2,8):
    print('Doing path {}'.format(l))
    pathres[l]=pathcheck(l,eps)

Doing path 7


100%|█████████████████████████████████████████| 300/300 [00:13<00:00, 22.60it/s]


In [5]:
for l in range(2,8):
    print('Doing crazy {}'.format(l))
    crazyres[l]=crazycheck(l,eps)

Doing crazy 7


100%|█████████████████████████████████████████| 300/300 [10:32<00:00,  2.11s/it]


In [6]:
for l in range(3,8,2):
    print('Doing twist {}'.format(l))
    twistres[l]=twistcheck(l,eps)

Doing twist 7


100%|█████████████████████████████████████████| 300/300 [01:01<00:00,  4.87it/s]


In [7]:
#save the data
with open('results/lengths/fidelity-pathres.pickle', 'wb') as file:
    pickle.dump(pathres, file)
with open('results/lengths/fidelity-twistres.pickle', 'wb') as file:
    pickle.dump(twistres, file)
with open('results/lengths/fidelity-crazyres.pickle', 'wb') as file:
    pickle.dump(crazyres, file)
